In [1]:
# # In case the Int Slider does not appear, run:
# !pip install ipywidgets
# !jupyter lab clean
# # restart kernel

In [2]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import os
import plotly.graph_objects as go
import plotly.colors as pc

In [3]:
# !pip list --not-required 

In [4]:
# Define column names for plot
columns_to_keep = ["Time [s]", "SoC [%]", "Physics-and-Transformer", "Physics", "Transformer"]    # All models results
# columns_to_keep = ["Time [s]", "SoC [%]", "Transformer"]    # Only Transformer results
# columns_to_keep = ["Time [s]", "SoC [%]", "Physics-and-Transformer", "Physics"]    # Physics and Transformers-And-Physics

# Define results for which dataset to display

In [5]:
# 1. Dataset: Full trips
dataset = "results/"

# List all experiment folders

In [ ]:
def list_directories(path):
    directories = [f"{path}/{d}" for d in os.listdir(path) if os.path.isdir(os.path.join(path, d)) 
                   and ".ipynb_checkpoints" not in d]
    return directories

In [7]:
experiments_paths = list_directories(dataset)
experiments_paths

['results//1-Delta approach, Lr=1e-05, epochs=200, dropout=0.1',
 'results//10-Data Loss + Physics Loss, lambda=0.5, Lr=0.0001, epochs=100, dropout=0.25',
 'results//11-Data Loss + Physics Loss, lambda=0.5, Lr=0.0001, epochs=100, dropout=0.1',
 'results//2-Delta approach, Lr=1e-05, epochs=100, dropout=0.2',
 'results//3-Delta approach, Lr=1e-05, epochs=100, dropout=0.1',
 'results//4-Delta approach, Lr=0.001, epochs=100, dropout=0.2',
 'results//5-Delta approach, Lr=0.001, epochs=100, dropout=0.1',
 'results//6-Delta approach, Lr=0.0001, epochs=100, dropout=0.2',
 'results//7-Delta approach, Lr=0.0001, epochs=100, dropout=0.1',
 'results//8-Data Loss + Physics Loss, lambda=10, Lr=0.0001, epochs=100, dropout=0.25',
 'results//9-Data Loss + Physics Loss, lambda=10, Lr=0.0001, epochs=100, dropout=0.1']

# Unify results of all models in each experiment

In [8]:
def get_filepaths_with_extension(file_extension=".csv", directory="."):
    output = []
    
    for file in os.listdir(directory):  
        # Check if the file has the required extension
        if file.endswith(file_extension):
            output.append(f"{directory}/{file}") 
            
    return output

## List all files from first model found

In [9]:
first_model_path = list_directories(experiments_paths[0])[0]
first_model_path += "/test_data_and_model_output"
first_model_path

'results//1-Delta approach, Lr=1e-05, epochs=200, dropout=0.1/Physics/test_data_and_model_output'

In [10]:
first_model_output_filenames = get_filepaths_with_extension(file_extension=".csv", directory=first_model_path)
first_model_output_filenames = [path.split("/")[-1] for path in first_model_output_filenames]    # keep only filename
first_model_output_filenames

['file-0.csv',
 'file-1.csv',
 'file-2.csv',
 'file-3.csv',
 'file-4.csv',
 'file-5.csv',
 'file-6.csv',
 'file-7.csv',
 'file-8.csv',
 'file-9.csv']

## Get and Concatenate results of all models of each experiment

In [11]:
dict_experiments_results = {}

In [12]:
for experiment_path in experiments_paths:
    # print(experiment_path)
    # get all model directories
    models_directories = list_directories(experiment_path)

    models_output_directories = [f"{model}/test_data_and_model_output" for model in models_directories]    # add directory where models store their output
    
    # get results from each model
    experiment_result = []
    for output_filename in first_model_output_filenames:    # iterate over each filename to make sure the comparison of the different models is correct
        df = None
        for models_output_directory in models_output_directories:    # iterate over the output directories of each model
            csv_filepath = f"{models_output_directory}/{output_filename}"
            df_new = pd.read_csv(csv_filepath, sep=";", encoding="ISO-8859-2")

            # if there is no dataframe stored yet
            if df is None:
                df = df_new.copy()
            else: 
                df = pd.concat([df, df_new], axis=1)    # concatenating along columns 

        # keep only important columns for plots
        df = df[columns_to_keep]    
        # remove duplicated columns
        df = df.loc[:,~df.columns.duplicated()]
        
        # print(df)
        experiment_result.append(df)

    experiment_name = experiment_path.split("/")[-1]
    dict_experiments_results[experiment_name] = experiment_result

In [13]:
dict_experiments_results

{'1-Delta approach, Lr=1e-05, epochs=200, dropout=0.1': [          Time [s]  SoC [%]  Physics-and-Transformer    Physics
  0        -0.000017     85.4                85.531641  85.400002
  1         0.099959     85.4                85.531566  85.399825
  2         0.199934     85.4                85.530805  85.399643
  3         0.300032     85.4                85.528599  85.399452
  4         0.400008     85.4                85.527251  85.399246
  ...            ...      ...                      ...        ...
  13620  1362.000000     71.5                71.481405  68.958252
  13621  1362.100000     71.5                71.488701  68.957993
  13622  1362.200000     71.5                71.496001  68.957733
  13623  1362.300000     71.5                71.488346  68.957466
  13624  1362.400000     71.5                71.480699  68.957199
  
  [13625 rows x 4 columns],
          Time [s]  SoC [%]  Physics-and-Transformer    Physics
  0      -0.000017     84.2                84.466001  84.1

# Create interactive plots

In [14]:
# !pip install plotly

In [15]:
model_output_column_names = ["Physics-and-Transformer", "Transformer", "Physics"]

In [16]:
def plot_soc(file_id, model_outputs_to_display):
    
    # Use Plotly's "Safe" qualitative color palette. Source of color pallete: https://plotly.com/python/discrete-color/
    colorblind_palette = pc.qualitative.Safe

    for experiment_name in list(dict_experiments_results.keys()):
        experiment_results = dict_experiments_results[experiment_name][file_id]
    
        # Create the figure
        fig = go.Figure()
        
        # Plot actual SoC only from first model
        fig.add_trace(go.Scatter(
            x=experiment_results['Time [s]'],
            y=experiment_results['SoC [%]'],
            mode='lines',
            line=dict(width=3, color=colorblind_palette[0]),
            name='Actual SoC [%]'
        ))

        marker_styles = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down']
        for idx_model, model_output_column_name in enumerate(model_output_column_names):
            # If model_output_column_name is not in selected model_outputs_to_display, skip
            if model_output_column_name not in model_outputs_to_display:
                continue     

            line_opacity = 0.7
            fig.add_trace(go.Scatter(
                x=experiment_results['Time [s]'],
                y=experiment_results[model_output_column_name],
                mode='lines',
                line=dict(dash='dot', width=3, 
                          color=f"rgba{tuple(map(int, colorblind_palette[(idx_model % len(colorblind_palette)) + 1][4:-1].split(',')))[:3] + (line_opacity,)}"),
                name=f"Model: {model_output_column_name}"
            ))    # color with opacity is last answer from AI assistant: https://chatgpt.com/share/67a8fb48-ae30-800b-b9c4-44caa979d630
            
        # Update layout
        fig.update_layout(
            title=f'Experiment: {experiment_name}<br><sup>Actual vs Estimated State of Charge (SoC %) of file {file_id}</sup>',
            xaxis_title='Time [s]',
            yaxis_title='State of Charge [%]',
            legend_title='Legend',
            template='plotly_white'
        )
        
        # Show the plot
        fig.show()

In [17]:
# get total number of files
total_files = len(dict_experiments_results[list(dict_experiments_results)[0]])    # number of dataframes in first experiments

In [18]:
# Create Slider for multiple selection of models
model_select_multiple = widgets.SelectMultiple(
    options=model_output_column_names,
    value=model_output_column_names,
    rows=len(model_output_column_names),
    description='Models:',
    disabled=False
)

# Create a slider for selecting the ID
id_slider = widgets.IntSlider(
    value=0, 
    min=0, 
    max=(total_files -1),    # max is the total number of file in any of the model results 
    step=1, 
    description='Test file ID:',
    continuous_update=False,
    style={'slider_button_width': '20px'}  # Enable built-in buttons
)

# Create a horizontal layout for the id_slider and model_select_multiple
controls = widgets.HBox([model_select_multiple, id_slider])

# Connect the slider to the plot function
interactive_plot = widgets.interactive_output(plot_soc, {'file_id': id_slider, 'model_outputs_to_display': model_select_multiple})

# Display the controls widgets and plot together
display(controls, interactive_plot)

Output()